## Titanic - Machine Learning from Disaster
### Start here! Predict survival on the Titanic and get familiar with ML basics

Dataset Description
-------------------

### Overview

The data has been split into two groups:

-   training set (train.csv)
-   test set (test.csv)

The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the "ground truth") for each passenger. Your model will be based on "features" like passengers' gender and class. You can also use [feature engineering ](https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/)to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

### Data Dictionary

| Variable | Definition | Key |
|---|---|---|
| survival | Survival | 0 = No, 1 = Yes |
| pclass | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |
| sex | Sex |  |
| Age | Age in years |  |
| sibsp | # of siblings / spouses aboard the Titanic |  |
| parch | # of parents / children aboard the Titanic |  |
| ticket | Ticket number |  |
| fare | Passenger fare |  |
| cabin | Cabin number |  |
| embarked | Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton |

### Variable Notes

pclass: A proxy for socio-economic status (SES)\
1st = Upper\
2nd = Middle\
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...\
Sibling = brother, sister, stepbrother, stepsister\
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...\
Parent = mother, father\
Child = daughter, son, stepdaughter, stepson\
Some children travelled only with a nanny, therefore parch=0 for them.

### SOLUTION
---

In [1]:
import pandas as pd


In [2]:
train_data = pd.read_csv("train.csv")
test_data  = pd.read_csv("test.csv")
test_ids = test_data["PassengerId"]

In [3]:
print("Train -> SIZE : ", train_data.size, "\tSHAPE : ", train_data.shape)
print("Test  -> SIZE : ", test_data.size,  "\tSHAPE : ", test_data.shape)

Train -> SIZE :  10692 	SHAPE :  (891, 12)
Test  -> SIZE :  4598 	SHAPE :  (418, 11)


In [4]:
train_data.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
435,436,1,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,B96 B98,S
250,251,0,3,"Reed, Mr. James George",male,NaN,0,0,362316,7.2500,NaN,S
633,634,0,1,"Parr, Mr. William Henry Marsh",male,NaN,0,0,112052,0.0000,NaN,S
723,724,0,2,"Hodges, Mr. Henry Price",male,50.0,0,0,250643,13.0000,NaN,S
277,278,0,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0000,NaN,S
689,690,1,1,"Madill, Miss. Georgette Alexandra",female,15.0,0,1,24160,211.3375,B5,S
173,174,0,3,"Sivola, Mr. Antti Wilhelm",male,21.0,0,0,STON/O 2. 3101280,7.9250,NaN,S
260,261,0,3,"Smith, Mr. Thomas",male,NaN,0,0,384461,7.7500,NaN,Q
449,450,1,1,"Peuchen, Major. Arthur Godfrey",male,52.0,0,0,113786,30.5000,C104,S
725,726,0,3,"Oreskovic, Mr. Luka",male,20.0,0,0,315094,8.6625,NaN,S


In [5]:
test_data.sample(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
390,1282,1,"Payne, Mr. Vivian Ponsonby",male,23.0,0,0,12749,93.5000,B24,S
78,970,2,"Aldworth, Mr. Charles Augustus",male,30.0,0,0,248744,13.0000,NaN,S
277,1169,2,"Faunthorpe, Mr. Harry",male,40.0,1,0,2926,26.0000,NaN,S
71,963,3,"Minkoff, Mr. Lazar",male,21.0,0,0,349211,7.8958,NaN,S
204,1096,2,"Andrew, Mr. Frank Thomas",male,25.0,0,0,C.A. 34050,10.5000,NaN,S
135,1027,3,"Carlsson, Mr. Carl Robert",male,24.0,0,0,350409,7.8542,NaN,S
85,977,3,"Khalil, Mr. Betros",male,NaN,1,0,2660,14.4542,NaN,C
133,1025,3,"Thomas, Mr. Charles P",male,NaN,1,0,2621,6.4375,NaN,C
10,902,3,"Ilieff, Mr. Ylio",male,NaN,0,0,349220,7.8958,NaN,S
94,986,1,"Birnbaum, Mr. Jakob",male,25.0,0,0,13905,26.0000,NaN,C


In [6]:
# Creating Function to Drop Columns from the DataFrame
# and Fill missing rows where data in not present

def dataset_cleaner(data):
    # Dropping Following columns as they will be unnecessary for predictions
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1)

    # Filling missing Values
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
    
    # U --> Unknown Token
    data.Embarked.fillna("U", inplace=True)

    return data


train_data = dataset_cleaner(train_data)
test_data  = dataset_cleaner(test_data)

In [7]:
print("Train -> SIZE : ", train_data.size, "\tSHAPE : ", train_data.shape)
print("Test  -> SIZE : ", test_data.size,  "\tSHAPE : ", test_data.shape)

Train -> SIZE :  7128 	SHAPE :  (891, 8)
Test  -> SIZE :  2926 	SHAPE :  (418, 7)


In [8]:
train_data.sample(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
683,0,3,male,14.0,5,2,46.9000,S
637,0,2,male,31.0,1,1,26.2500,S
501,0,3,female,21.0,0,0,7.7500,Q
480,0,3,male,9.0,5,2,46.9000,S
508,0,3,male,28.0,0,0,22.5250,S
550,1,1,male,17.0,0,2,110.8833,C
582,0,2,male,54.0,0,0,26.0000,S
752,0,3,male,33.0,0,0,9.5000,S
526,1,2,female,50.0,0,0,10.5000,S
581,1,1,female,39.0,1,1,110.8833,C


In [9]:
test_data.sample(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
333,3,female,19.0,1,1,15.7417,C
195,3,male,33.0,0,0,7.8542,S
115,3,male,18.0,1,0,14.4542,C
348,2,male,24.0,0,0,13.5000,S
406,2,male,23.0,1,0,10.5000,S
114,1,female,63.0,1,0,221.7792,S
279,2,male,22.0,0,0,10.5000,S
109,2,male,18.5,0,0,13.0000,S
13,2,male,63.0,1,0,26.0000,S
5,3,male,14.0,0,0,9.2250,S


In [10]:
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()

def encoder(data):
    # Encoding the following Columns values to encoded integers
    cols = ["Sex", "Embarked"]

    for col in cols:
        data[col] = lab_enc.fit_transform(data[col])
    

encoder(train_data)
encoder(test_data)        

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x = train_data.drop("Survived", axis=1)
y = train_data["Survived"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
classifier = LogisticRegression(random_state=3, max_iter=9000)
classifier = classifier.fit(x_train, y_train)

pred = classifier.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score

print("Classifier Accuracy :",accuracy_score(y_test, pred))

Classifier Accuracy : 0.8100558659217877


In [14]:
submit_pred = classifier.predict(test_data)

submit_df = pd.DataFrame({"PassengerId":test_ids.values,
                        "Survived": submit_pred})

submit_df.to_csv("submission.csv", index=False)